In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, confusion_matrix
import time

In [3]:
# TensorFlow sürümünü yazdır
print(f"TensorFlow Sürümü: {tf.__version__}")

TensorFlow Sürümü: 2.18.0


In [5]:
# TensorFlow hafıza yönetimi
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [7]:
# Sabit parametreler
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 20  # Epoch sayısını azalttık, daha hızlı eğitim için
PATIENCE = 5  # Erken durdurma için sabır parametresi

In [9]:
# Veri seti seçimi ve yollarını güncelleme
# MVTec veri seti yolu
MVTEC_DIR = r"C:\Users\Tansu\OneDrive\Masaüstü\mvtec"  # MVTec veri seti yolu - kendi yolunuzu belirtin
#"C:\Users\Tansu\OneDrive\Masaüstü\mvtec"

In [11]:
# VisA veri seti yolu - kendi yolunuzu belirtin
VISA_DIR = r"C:\Users\Tansu\OneDrive\Masaüstü\visa"  # VisA veri seti yolu - kendi yolunuzu belirtin
#"C:\Users\Tansu\OneDrive\Masaüstü\visa"

In [13]:
# Sonuçları saklamak için dizin
RESULTS_DIR = "anomaly_detection_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

In [15]:
# Veri setleri ve kategorileri
MVTEC_CATEGORIES = ["hazelnut", "screw", "pill", "toothbrush", "bottle", "cable", "capsule", "carpet", "grid", "tile", "wood"]
VISA_CATEGORIES = ["candle", "capsules", "cashew", "chewinggum", "fryum", "macaroni1", "macaroni2", "pcb1", "pcb2", "pcb3", "pipe_fryum"]

In [17]:
# Hangi veri setlerini kullanacağız
USE_MVTEC = True  # MVTec veri setini kullan
USE_VISA = True   # VisA veri setini kullan

In [19]:
# Görüntüyü yükleyen yardımcı fonksiyon - PIL kullanıyor
def load_image(file_path, target_size=(IMG_SIZE, IMG_SIZE)):
    """PIL kullanarak görüntüyü yükler ve numpy dizisine dönüştürür"""
    try:
        # Görüntüyü PIL ile yükle
        img = Image.open(file_path)
        # Gri tonlamaya dönüştür
        img = img.convert('L')
        # Yeniden boyutlandır
        img = img.resize(target_size)
        # Numpy dizisine dönüştür
        img_array = np.array(img)
        return img_array
    except Exception as e:
        print(f"UYARI: Görüntü yüklenemedi '{file_path}': {e}")
        return None

In [21]:
# MVTec Veri Seti için veri yükleme fonksiyonları
def load_mvtec_data(data_dir, category, img_size=IMG_SIZE):
    """MVTec veri setinden belirli bir kategorideki görüntüleri yükler"""
    train_dir = os.path.join(data_dir, category, "train")
    test_dir = os.path.join(data_dir, category, "test")
    
    print(f"Eğitim dizini: {train_dir}")
    print(f"Test dizini: {test_dir}")
    
    # Dizinlerin varlığını kontrol et
    if not os.path.exists(train_dir):
        raise FileNotFoundError(f"Eğitim dizini bulunamadı: {train_dir}")
    if not os.path.exists(test_dir):
        raise FileNotFoundError(f"Test dizini bulunamadı: {test_dir}")
    
    # Eğitim verilerini yükle (sadece normal örnekler)
    train_good_dir = os.path.join(train_dir, "good")
    
    if not os.path.exists(train_good_dir):
        raise FileNotFoundError(f"Eğitim 'good' dizini bulunamadı: {train_good_dir}")
    
    print(f"Eğitim 'good' dizini: {train_good_dir}")
    print(f"Eğitim 'good' dizinindeki dosya sayısı: {len(os.listdir(train_good_dir))}")
    
    train_images = []
    for img_name in tqdm(os.listdir(train_good_dir), desc=f"{category}: Eğitim verilerini yükleme"):
        img_path = os.path.join(train_good_dir, img_name)
        img = load_image(img_path)
        if img is not None:
            train_images.append(img)
    
    # Test verilerini ve etiketleri yükle
    test_images = []
    test_labels = []  # 0: normal, 1: anomali
    test_paths = []  # Dosya yollarını saklamak için
    test_types = []  # Kusur tiplerini saklamak için
    
    # Normal test örnekleri
    normal_test_dir = os.path.join(test_dir, "good")
    if os.path.exists(normal_test_dir):
        print(f"Test 'good' dizini: {normal_test_dir}")
        print(f"Test 'good' dizinindeki dosya sayısı: {len(os.listdir(normal_test_dir))}")
        
        for img_name in tqdm(os.listdir(normal_test_dir), desc=f"{category}: Normal test verilerini yükleme"):
            img_path = os.path.join(normal_test_dir, img_name)
            img = load_image(img_path)
            if img is not None:
                test_images.append(img)
                test_labels.append(0)  # Normal örnek
                test_paths.append(img_path)
                test_types.append("good")
    else:
        print(f"UYARI: Normal test dizini bulunamadı: {normal_test_dir}")
    
    # Anomali test örnekleri
    defect_types = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d)) and d != "good"]
    print(f"Tespit edilen kusur tipleri: {defect_types}")
    
    for defect_type in defect_types:
        defect_dir = os.path.join(test_dir, defect_type)
        if os.path.isdir(defect_dir):
            print(f"Kusur dizini: {defect_dir}")
            print(f"Kusur dizinindeki dosya sayısı: {len(os.listdir(defect_dir))}")
            
            for img_name in tqdm(os.listdir(defect_dir), desc=f"{category}: {defect_type} verilerini yükleme"):
                img_path = os.path.join(defect_dir, img_name)
                img = load_image(img_path)
                if img is not None:
                    test_images.append(img)
                    test_labels.append(1)  # Anomali örneği
                    test_paths.append(img_path)
                    test_types.append(defect_type)

    # Eğer görüntü yoksa uyarı ver
    if len(train_images) == 0:
        print("UYARI: Hiçbir eğitim görüntüsü yüklenemedi!")
        return [], [], [], [], []
        
    if len(test_images) == 0:
        print("UYARI: Hiçbir test görüntüsü yüklenemedi!")
        return np.array(train_images), [], [], [], []
    
    # NumPy dizilerine dönüştür
    train_images = np.array(train_images)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    # Veri şekillerini yazdır
    print(f"Eğitim veriseti: {train_images.shape} - Min: {train_images.min()}, Max: {train_images.max()}")
    print(f"Test veriseti: {test_images.shape} - Min: {test_images.min()}, Max: {test_images.max()}")
    print(f"Test etiketleri: {test_labels.shape}")
    print(f"Anomali sayısı: {np.sum(test_labels == 1)}")
    print(f"Normal test örneği sayısı: {np.sum(test_labels == 0)}")
    
    return train_images, test_images, test_labels, test_paths, test_types


In [23]:
def load_visa_data(data_dir, category, img_size=IMG_SIZE):
    """
    VisA veri setinden belirli bir kategorideki görüntüleri yükler
    Dizin yapısı:
    - category/
      - data/
        - images/
          - normal/
          - anomaly/
    """
    # VisA veri seti için dizin yapısı
    category_dir = os.path.join(data_dir, category)
    images_dir = os.path.join(category_dir, "data", "images")
    normal_dir = os.path.join(images_dir, "normal")
    anomaly_dir = os.path.join(images_dir, "anomaly")
    
    print(f"VisA kategori dizini: {category_dir}")
    print(f"VisA görüntü dizini: {images_dir}")
    print(f"VisA normal görüntü dizini: {normal_dir}")
    print(f"VisA anomali görüntü dizini: {anomaly_dir}")
    
    # Dizinlerin varlığını kontrol et
    if not os.path.exists(category_dir):
        raise FileNotFoundError(f"VisA kategori dizini bulunamadı: {category_dir}")
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"VisA görüntü dizini bulunamadı: {images_dir}")
    if not os.path.exists(normal_dir):
        raise FileNotFoundError(f"VisA normal dizini bulunamadı: {normal_dir}")
    if not os.path.exists(anomaly_dir):
        print(f"UYARI: VisA anomali dizini bulunamadı: {anomaly_dir}")
    
    # Normal görüntüleri yükle (eğitim için)
    train_images = []
    for img_name in tqdm(os.listdir(normal_dir), desc=f"{category} (VisA): Normal görüntüleri yükleme"):
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(normal_dir, img_name)
        img = load_image(img_path)
        if img is not None:
            train_images.append(img)
    
    if len(train_images) == 0:
        print(f"UYARI: VisA için hiçbir normal görüntü yüklenemedi!")
        return [], [], [], [], []
    
    # Normal ve anomali görüntülerini test setine ekle
    # Tüm veriyi karıştır ve %70 eğitim, %30 test olacak şekilde ayır
    test_images = []
    test_labels = []
    test_paths = []
    test_types = []
    
    # Normal görüntülerin bir kısmını test setine ayır
    np.random.shuffle(train_images)
    test_size = int(len(train_images) * 0.3)
    test_normal_images = train_images[-test_size:]
    train_images = train_images[:-test_size]
    
    # Normal görüntüleri test setine ekle
    for img in test_normal_images:
        test_images.append(img)
        test_labels.append(0)  # Normal
        test_paths.append("")
        test_types.append("normal")
    
    # Anomali görüntülerini yükle
    if os.path.exists(anomaly_dir):
        anomaly_images = []
        for img_name in tqdm(os.listdir(anomaly_dir), desc=f"{category} (VisA): Anomali görüntüleri yükleme"):
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            img_path = os.path.join(anomaly_dir, img_name)
            img = load_image(img_path)
            if img is not None:
                test_images.append(img)
                test_labels.append(1)  # Anomali
                test_paths.append(img_path)
                test_types.append("anomaly")
    else:
        print(f"UYARI: VisA anomali dizini bulunamadı, testler sadece normal örneklerle yapılacak.")
    
    # NumPy dizilerine dönüştür
    train_images = np.array(train_images)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    # Veri şekillerini yazdır
    print(f"VisA Eğitim veriseti: {train_images.shape} - Min: {train_images.min()}, Max: {train_images.max()}")
    print(f"VisA Test veriseti: {test_images.shape} - Min: {test_images.min()}, Max: {test_images.max()}")
    print(f"VisA Test etiketleri: {test_labels.shape}")
    print(f"VisA Anomali sayısı: {np.sum(test_labels == 1)}")
    print(f"VisA Normal test örneği sayısı: {np.sum(test_labels == 0)}")
    
    return train_images, test_images, test_labels, test_paths, test_types


In [25]:
def preprocess_data(train_images, test_images):
    """Verileri normalleştirme ve eğitim için hazırlama"""
    # Kanalı ekle (gri tonlamalı görüntüler)
    train_images = np.expand_dims(train_images, axis=-1)
    test_images = np.expand_dims(test_images, axis=-1)
    
    # Verileri [0,1] aralığına normalleştir
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    
    return train_images, test_images

In [27]:
def create_autoencoder(input_shape):
    """Autoencoder modeli oluşturma"""
    # Model giriş boyutu
    input_img = Input(shape=input_shape, name='input_layer')
    
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)  # 64x64
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)  # 32x32
    
    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # 64x64
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)  # 128x128
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    # Model oluşturma
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder

In [29]:
def detect_anomalies(test_images, test_labels, model, threshold_percentile=95):
    """Test görüntülerinde anomali tespiti yapma"""
    # Rekonstrüksiyon yapma
    print("Rekonstrüksiyon başlatılıyor...")
    reconstructed = model.predict(test_images)
    
    # Rekonstrüksiyon hatası hesaplama (piksellerin karelerinin ortalaması)
    mse = np.mean(np.square(test_images - reconstructed), axis=(1, 2, 3))
    
    # Eşik değeri belirleme
    threshold = np.percentile(mse, threshold_percentile)
    print(f"Anomali eşik değeri ({threshold_percentile}. yüzdelik): {threshold:.6f}")
    
    # Anomali tahminleri
    predictions = (mse > threshold).astype(int)
    
    # Tahmin sonuçlarını değerlendirme
    accuracy = np.mean(predictions == test_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(test_labels, predictions, average='binary')
    
    # ROC eğrisi
    fpr, tpr, _ = roc_curve(test_labels, mse)
    roc_auc = auc(fpr, tpr)
    
    # Sonuçları yazdırma
    print(f"Doğruluk (Accuracy): {accuracy:.4f}")
    print(f"Hassasiyet (Precision): {precision:.4f}")
    print(f"Duyarlılık (Recall): {recall:.4f}")
    print(f"F1 Skoru: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    
    return reconstructed, mse, predictions, threshold, accuracy, precision, recall, f1, roc_auc

In [31]:
def visualize_reconstruction_results(test_images, test_labels, reconstructed, mse, predictions, threshold, 
                                     test_types, category, output_dir=RESULTS_DIR, num_samples=10, dataset_type="mvtec"):
    """
    Autoencoder modelinin rekonstrüksiyon sonuçlarını görselleştirir
    """
    # Normal ve anomali örneklerini ayrı dizinlere ayır
    normal_indices = np.where(test_labels == 0)[0]
    anomaly_indices = np.where(test_labels == 1)[0]
    
    if len(normal_indices) == 0 or len(anomaly_indices) == 0:
        print(f"UYARI: {category} için yeterli normal veya anomali örneği bulunamadı.")
        return
    
    # Normal ve anomali örneklerinden rastgele seç
    np.random.shuffle(normal_indices)
    np.random.shuffle(anomaly_indices)
    
    # Kaç örnek gösterileceğini belirle
    n_normal = min(num_samples // 2, len(normal_indices))
    n_anomaly = min(num_samples - n_normal, len(anomaly_indices))
    
    # Örnekleri seç
    selected_normal = normal_indices[:n_normal]
    selected_anomaly = anomaly_indices[:n_anomaly]
    
    # Tüm seçilen örnekleri birleştir
    selected_indices = np.concatenate([selected_normal, selected_anomaly])
    np.random.shuffle(selected_indices)
    
    # Görselleştirme
    n_samples = len(selected_indices)
    fig, axes = plt.subplots(n_samples, 3, figsize=(12, n_samples * 4))
    fig.suptitle(f"{category} ({dataset_type}) - Orijinal, Rekonstrüksiyon ve Fark - Eşik: {threshold:.6f}", fontsize=16)
    
    # Tek örnek durumu için
    if n_samples == 1:
        axes = np.array([axes])
    
    for i, idx in enumerate(selected_indices):
        # Orijinal görüntü
        axes[i, 0].imshow(test_images[idx].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
        type_label = test_types[idx] if test_types and idx < len(test_types) else "?"
        axes[i, 0].set_title(f"Orijinal - {'Anomali: ' + type_label if test_labels[idx] else 'Normal'}")
        axes[i, 0].axis('off')
        
        # Rekonstrüksiyon
        axes[i, 1].imshow(reconstructed[idx].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
        axes[i, 1].set_title(f"Rekonstrüksiyon - MSE: {mse[idx]:.6f}")
        axes[i, 1].axis('off')
        
        # Fark (mutlak)
        diff = np.abs(test_images[idx] - reconstructed[idx]).reshape(IMG_SIZE, IMG_SIZE)
        axes[i, 2].imshow(diff, cmap='hot', vmin=0, vmax=np.max(diff) if np.max(diff) > 0 else 1)
        
        # Modelin tahmini doğru mu?
        is_correct = predictions[idx] == test_labels[idx]
        result_color = "green" if is_correct else "red"
        
        axes[i, 2].set_title(f"Fark - Tahmin: {'Anomali' if predictions[idx] else 'Normal'}", 
                             color=result_color)
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_reconstruction_results.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Rekonstrüksiyon sonuçları kaydedildi: {output_path}")

def compare_normal_vs_anomaly(test_images, test_labels, reconstructed, mse, threshold, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """Normal ve anomalili görüntülerin rekonstrüksiyon hatalarını karşılaştırır"""
    
    # Normal ve anomali örneklerini ayır
    normal_mse = mse[test_labels == 0]
    anomaly_mse = mse[test_labels == 1]
    
    if len(normal_mse) == 0 or len(anomaly_mse) == 0:
        print(f"UYARI: {category} için yeterli normal veya anomali MSE değeri bulunamadı.")
        return
    
    # İstatistiksel bilgiler
    print(f"Normal örnekler MSE - Min: {normal_mse.min():.6f}, Max: {normal_mse.max():.6f}, Ortalama: {normal_mse.mean():.6f}")
    print(f"Anomali örnekler MSE - Min: {anomaly_mse.min():.6f}, Max: {anomaly_mse.max():.6f}, Ortalama: {anomaly_mse.mean():.6f}")
    
    # Histogram karşılaştırması
    plt.figure(figsize=(12, 6))
    plt.hist(normal_mse, bins=30, alpha=0.5, label="Normal", color="green")
    plt.hist(anomaly_mse, bins=30, alpha=0.5, label="Anomali", color="red")
    plt.axvline(x=threshold, color='blue', linestyle='--', label=f"Eşik ({threshold:.6f})")
    plt.title(f"{category} ({dataset_type}) - Normal ve Anomali Örneklerinin Rekonstrüksiyon Hatası Dağılımı")
    plt.xlabel("Rekonstrüksiyon Hatası (MSE)")
    plt.ylabel("Frekans")
    plt.legend()
    plt.grid(True, alpha=0.3)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_mse_comparison_histogram.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"MSE karşılaştırma histogramı kaydedildi: {output_path}")

def visualize_roc_curve(test_labels, mse, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """ROC eğrisini çizer"""
    # ROC eğrisi
    fpr, tpr, _ = roc_curve(test_labels, mse)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{category} ({dataset_type}) - Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_roc_curve.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"ROC eğrisi kaydedildi: {output_path}")

def visualize_confusion_matrix(test_labels, predictions, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """Karışıklık matrisini görselleştirir"""
    # Karışıklık matrisi
    cm = confusion_matrix(test_labels, predictions)
    
    # Normalize et
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Görselleştirme
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'{category} ({dataset_type}) - Karışıklık Matrisi')
    plt.colorbar()
    
    # Etiketler
    classes = ['Normal', 'Anomali']
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    
    # Sayıları ve yüzdeleri göster
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, f"{cm[i, j]}\n({cm_norm[i, j]*100:.1f}%)",
                     horizontalalignment='center',
                     color='white' if cm[i, j] > thresh else 'black')
    
    plt.tight_layout()
    plt.ylabel('Gerçek Etiket')
    plt.xlabel('Tahmin Edilen Etiket')
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_confusion_matrix.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Karışıklık matrisi kaydedildi: {output_path}")

def visualize_detailed_comparison(test_images, test_labels, reconstructed, mse, predictions, threshold, 
                                 test_types, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """
    Her kusur tipi için ayrıntılı karşılaştırma yapar:
    1. Orijinal görüntü
    2. Rekonstrüksiyon
    3. Fark haritası
    4. Hata değerlerini görselleştirme
    """
    # Kusur tiplerine göre gruplama
    unique_types = set([t for t in test_types if t != "good" and t != "normal"])
    
    if not unique_types:
        print(f"UYARI: {category} için anomali tipi bulunamadı.")
        return
    
    # Ayrı bir görselleştirme penceresi oluştur
    for defect_type in unique_types:
        # Bu kusur tipine ait örneklerin indekslerini bul
        indices = [i for i, t in enumerate(test_types) if t == defect_type]
        
        if not indices:
            continue
            
        # En fazla 3 örnek göster
        num_samples = min(3, len(indices))
        selected_indices = np.random.choice(indices, num_samples, replace=False)
        
        fig, axes = plt.subplots(num_samples, 4, figsize=(16, num_samples * 4))
        fig.suptitle(f"{category} ({dataset_type}) - {defect_type} Kusur Tipi Analizi", fontsize=16)
        
        for i, idx in enumerate(selected_indices):
            # Tek örnek durumunda axes'i 2B hale getir
            if num_samples == 1:
                ax_row = axes
            else:
                ax_row = axes[i]
            
            # 1. Orijinal görüntü
            ax_row[0].imshow(test_images[idx].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
            ax_row[0].set_title(f"Orijinal - {defect_type}")
            ax_row[0].axis('off')
            
            # 2. Rekonstrüksiyon
            ax_row[1].imshow(reconstructed[idx].reshape(IMG_SIZE, IMG_SIZE), cmap='gray')
            ax_row[1].set_title(f"Rekonstrüksiyon")
            ax_row[1].axis('off')
            
            # 3. Fark (mutlak)
            diff = np.abs(test_images[idx] - reconstructed[idx]).reshape(IMG_SIZE, IMG_SIZE)
            im = ax_row[2].imshow(diff, cmap='hot', vmin=0, vmax=np.max(diff) if np.max(diff) > 0 else 1)
            ax_row[2].set_title(f"Fark Haritası")
            ax_row[2].axis('off')
            
            # 4. Hata değeri ve eşik
            error_bar = np.array([0, mse[idx], threshold, threshold * 1.5])  # Hata değerleri
            bar_colors = ['white', 'blue' if mse[idx] < threshold else 'red', 'green', 'white']
            
            ax_row[3].barh(['', 'MSE', 'Eşik', ''], error_bar, color=bar_colors)
            ax_row[3].axvline(x=threshold, color='green', linestyle='--')
            ax_row[3].set_title(f"MSE: {mse[idx]:.6f}, Eşik: {threshold:.6f}")
            ax_row[3].set_xlim(0, max(threshold * 2, mse[idx] * 1.2))
            
            # Tahmin sonucunu göster
            prediction_result = "Normal" if predictions[idx] == 0 else "Anomali"
            ground_truth = "Anomali" if test_labels[idx] == 1 else "Normal"
            is_correct = predictions[idx] == test_labels[idx]
            result_color = "green" if is_correct else "red"
            
            ax_row[3].text(threshold / 2, 0, 
                          f"Tahmin: {prediction_result} (Gerçek: {ground_truth})", 
                          color=result_color, fontsize=10, ha='center')
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        output_path = os.path.join(output_dir, f"{category}_{dataset_type}_{defect_type}_detailed_analysis.png")
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"{defect_type} kusur tipi detaylı analizi kaydedildi: {output_path}")

def visualize_all_defect_stats(test_images, test_labels, mse, predictions, threshold, test_types, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """
    Her kusur tipi için doğruluk istatistiklerini hesaplar ve görselleştirir
    """
    # Kusur tiplerine göre gruplama
    unique_types = set(test_types)
    
    if not unique_types:
        print(f"UYARI: {category} için kusur tipi bulunamadı.")
        return
    
    # Her kusur tipi için metrikler
    type_metrics = {}
    
    for defect_type in unique_types:
        # Bu tipe ait örneklerin indeksleri
        indices = [i for i, t in enumerate(test_types) if t == defect_type]
        
        if not indices:
            continue
        
        # Bu tipe ait gerçek etiketler ve tahminler
        type_labels = [test_labels[i] for i in indices]
        type_predictions = [predictions[i] for i in indices]
        type_mse = [mse[i] for i in indices]
        
        # Metrikleri hesapla
        type_accuracy = np.mean(np.array(type_predictions) == np.array(type_labels))
        
        # Hata istatistikleri
        mse_min = np.min(type_mse)
        mse_max = np.max(type_mse)
        mse_mean = np.mean(type_mse)
        mse_std = np.std(type_mse)
        
        # Eşiğe göre doğru tahmin oranı
        if defect_type == "good" or defect_type == "normal":
            # Normal örnekler için doğru tahmin oranı (MSE < threshold)
            correct_ratio = np.mean(np.array(type_mse) < threshold)
        else:
            # Anomali örnekleri için doğru tahmin oranı (MSE > threshold)
            correct_ratio = np.mean(np.array(type_mse) > threshold)
        
        type_metrics[defect_type] = {
            'accuracy': type_accuracy,
            'mse_min': mse_min,
            'mse_max': mse_max,
            'mse_mean': mse_mean,
            'mse_std': mse_std,
            'correct_ratio': correct_ratio,
            'count': len(indices)
        }
    
    # Metrik tablosunu görselleştir
    fig, ax = plt.subplots(figsize=(12, len(unique_types) * 1 + 2))
    fig.suptitle(f"{category} ({dataset_type}) - Kusur Tiplerine Göre Performans Metrikleri", fontsize=16)
    
    # Tablo verilerini hazırla
    types = list(unique_types)
    counts = [type_metrics[t]['count'] for t in types]
    accuracies = [type_metrics[t]['accuracy'] * 100 for t in types]
    correct_ratios = [type_metrics[t]['correct_ratio'] * 100 for t in types]
    mse_means = [type_metrics[t]['mse_mean'] for t in types]
    
    # Tablo oluştur
    table_data = [
        [t, f"{c}", f"{a:.1f}%", f"{r:.1f}%", f"{m:.6f}"] 
        for t, c, a, r, m in zip(types, counts, accuracies, correct_ratios, mse_means)
    ]
    
    colLabels = ["Kusur Tipi", "Örnek Sayısı", "Doğruluk", "Eşik Doğruluğu", "Ort. MSE"]
    
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=table_data,
                     colLabels=colLabels,
                     loc='center',
                     cellLoc='center')
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Başlık hücreleri için stili ayarla
    for i in range(len(colLabels)):
        table._cells[(0, i)].set_text_props(weight='bold')
    
    plt.tight_layout()
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_defect_type_metrics.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Kusur tipi metrikleri kaydedildi: {output_path}")
    
    # MSE dağılımlarını kusur tipine göre görselleştir
    plt.figure(figsize=(14, 8))
    
    for defect_type in unique_types:
        # Bu tipe ait MSE değerleri
        indices = [i for i, t in enumerate(test_types) if t == defect_type]
        type_mse = [mse[i] for i in indices]
        
        plt.hist(type_mse, bins=20, alpha=0.5, label=f"{defect_type} (n={len(indices)})")
    
    plt.axvline(x=threshold, color='black', linestyle='--', label=f"Eşik ({threshold:.6f})")
    plt.title(f"{category} ({dataset_type}) - Kusur Tiplerine Göre MSE Dağılımı")
    plt.xlabel("Rekonstrüksiyon Hatası (MSE)")
    plt.ylabel("Frekans")
    plt.legend()
    plt.grid(True, alpha=0.3)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_defect_type_mse_distribution.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Kusur tipine göre MSE dağılımı kaydedildi: {output_path}")

def visualize_all_categories_comparison(all_results, output_dir=RESULTS_DIR, output_path=None, title=None):
    """Tüm kategorilerin performans metriklerini karşılaştıran bir görselleştirme yapar"""
    
    if not all_results:
        print("Karşılaştırma için yeterli sonuç yok.")
        return
    
    # Kategorileri ve metrikleri dizilere dönüştür
    categories = [r['category'] for r in all_results]
    dataset_types = [r.get('dataset_type', 'unknown') for r in all_results]
    accuracy = [r['accuracy'] for r in all_results]
    precision = [r['precision'] for r in all_results]
    recall = [r['recall'] for r in all_results]
    f1 = [r['f1'] for r in all_results]
    roc_auc = [r['roc_auc'] for r in all_results]
    
    # Varsayılan başlık
    if title is None:
        title = 'Tüm Kategoriler için Performans Metrikleri Karşılaştırması'
    
    # Performans metriklerini görselleştir
    plt.figure(figsize=(14, 8))
    bar_width = 0.15
    x = np.arange(len(categories))
    
    # Her metrik için bir çubuk çiz
    plt.bar(x - 2*bar_width, accuracy, width=bar_width, label='Doğruluk', color='royalblue')
    plt.bar(x - bar_width, precision, width=bar_width, label='Hassasiyet', color='lightcoral')
    plt.bar(x, recall, width=bar_width, label='Duyarlılık', color='lightgreen')
    plt.bar(x + bar_width, f1, width=bar_width, label='F1 Skoru', color='gold')
    plt.bar(x + 2*bar_width, roc_auc, width=bar_width, label='ROC AUC', color='mediumpurple')
    
    plt.xlabel('Kategori')
    plt.ylabel('Değer')
    plt.title(title)
    
    # Kategori isimleri ve veri seti türü
    category_labels = [f"{cat}\n({dtype.upper()})" for cat, dtype in zip(categories, dataset_types)]
    plt.xticks(x, category_labels, rotation=45, ha='right')
    
    plt.legend()
    plt.ylim(0, 1.1)  # Metrikler 0-1 arası olduğu için
    plt.grid(True, alpha=0.3)
    
    # Değerleri grafiğin üzerine yaz
    for i in range(len(categories)):
        plt.text(i - 2*bar_width, accuracy[i] + 0.02, f"{accuracy[i]:.2f}", ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i - bar_width, precision[i] + 0.02, f"{precision[i]:.2f}", ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i, recall[i] + 0.02, f"{recall[i]:.2f}", ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + bar_width, f1[i] + 0.02, f"{f1[i]:.2f}", ha='center', va='bottom', fontsize=8, rotation=90)
        plt.text(i + 2*bar_width, roc_auc[i] + 0.02, f"{roc_auc[i]:.2f}", ha='center', va='bottom', fontsize=8, rotation=90)
    
    plt.tight_layout()
    
    # Çıktı dosya yolu
    if output_path is None:
        output_path = os.path.join(output_dir, 'all_categories_performance_comparison.png')
    
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Performans karşılaştırması kaydedildi: {output_path}")

In [33]:
def visualize_samples(image_dir, title="Görüntü Örnekleri", output_dir=RESULTS_DIR, rows=3, cols=4, sample_size=None):
    """Belirtilen dizindeki görüntülerden örnekleri görselleştirir"""
    
    # Dizindeki tüm görüntü dosyalarını al
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print(f"UYARI: '{image_dir}' içinde görüntü dosyası bulunamadı.")
        return
    
    # Rastgele örnek seç
    if sample_size is None:
        sample_size = rows * cols
    
    # Dizinde yeterli görüntü yoksa, tümünü kullan
    if len(image_files) < sample_size:
        sample_size = len(image_files)
        print(f"UYARI: Dizinde sadece {sample_size} adet görüntü var. Tümü kullanılacak.")
    
    # Görüntüleri karıştır ve belirlenen sayıda örnek seç
    random.shuffle(image_files)
    sampled_files = image_files[:sample_size]
    
    # Görselleştirme ızgarasını hazırla
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    fig.suptitle(title, fontsize=16)
    
    # Izgara tek boyutlu olabilir
    if rows == 1 and cols == 1:
        axes = np.array([axes])
    elif rows == 1 or cols == 1:
        axes = axes.flatten()
    
    # Seçilen görüntüleri yükle ve görselleştir
    for i, img_file in enumerate(sampled_files):
        if i >= rows * cols:
            break
            
        img_path = os.path.join(image_dir, img_file)
        img = load_image(img_path)
        
        if img is not None:
            row_idx = i // cols
            col_idx = i % cols
            
            if rows == 1 or cols == 1:
                axes[i].imshow(img, cmap='gray')
                axes[i].set_title(img_file)
                axes[i].axis('off')
            else:
                axes[row_idx, col_idx].imshow(img, cmap='gray')
                axes[row_idx, col_idx].set_title(img_file)
                axes[row_idx, col_idx].axis('off')
    
    # Boş kalan hücreleri gizle
    for i in range(len(sampled_files), rows * cols):
        row_idx = i // cols
        col_idx = i % cols
        
        if rows == 1 or cols == 1:
            axes[i].axis('off')
        else:
            axes[row_idx, col_idx].axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    
    # Görüntüyü kaydet
    safe_title = title.replace(" ", "_").replace(":", "").replace("-", "_")
    output_path = os.path.join(output_dir, f"{safe_title}.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Görselleştirme kaydedildi: {output_path}")

In [35]:
def visualize_defect_types_comparison(data_dir, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """Farklı kusur tiplerinin örnek görüntülerini yan yana karşılaştırır"""
    
    test_dir = os.path.join(data_dir, category, "test")
    
    # Normal örnekler dizini (dataset_type'a göre ayarlama)
    if dataset_type == "mvtec":
        normal_dir = os.path.join(test_dir, "good")
    else:  # VisA
        normal_dir = os.path.join(test_dir, "normal")
        if not os.path.exists(normal_dir):
            normal_dir = os.path.join(test_dir, "good")
    
    if not os.path.exists(normal_dir):
        print(f"UYARI: '{normal_dir}' dizini bulunamadı. Normal örnekler için test dizini kullanılıyor.")
        normal_dir = test_dir
    
    # Kusur tipleri
    if dataset_type == "mvtec":
        defect_types = [d for d in os.listdir(test_dir) 
                        if os.path.isdir(os.path.join(test_dir, d)) and d != "good"]
    else:  # VisA
        # VisA'da anomali dizinleri "anomaly", "defect" vb. olabilir
        anomaly_dirs = ["anomaly", "defect", "defective", "abnormal"]
        defect_types = []
        
        for anomaly_dir_name in anomaly_dirs:
            anomaly_dir = os.path.join(test_dir, anomaly_dir_name)
            if os.path.exists(anomaly_dir) and os.path.isdir(anomaly_dir):
                # Alt dizinleri kontrol et (kusur tipleri olabilir)
                sub_dirs = [d for d in os.listdir(anomaly_dir) 
                           if os.path.isdir(os.path.join(anomaly_dir, d))]
                if sub_dirs:
                    # Alt dizinler var, bunlar kusur tipleri
                    for sub_dir in sub_dirs:
                        defect_types.append(os.path.join(anomaly_dir_name, sub_dir))
                else:
                    # Alt dizin yok, doğrudan anomaly dizini
                    defect_types.append(anomaly_dir_name)
    
    if not defect_types:
        print(f"UYARI: '{category}' için kusur tipi bulunamadı.")
        return
    
    # Her kusur tipinden rasgele bir örnek seçelim
    fig, axes = plt.subplots(1, len(defect_types) + 1, figsize=(len(defect_types) * 4 + 4, 4))
    fig.suptitle(f"{category} ({dataset_type}) - Kusur Tipleri Karşılaştırması", fontsize=16)
    
    # Normal örnek
    normal_images = [f for f in os.listdir(normal_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if normal_images:
        normal_img_path = os.path.join(normal_dir, random.choice(normal_images))
        normal_img = load_image(normal_img_path)
        
        axes[0].imshow(normal_img, cmap='gray')
        axes[0].set_title("Normal (Good)")
        axes[0].axis('off')
    else:
        print(f"UYARI: '{normal_dir}' içinde normal görüntü bulunamadı.")
    
    # Her kusur tipinden bir örnek
    for i, defect in enumerate(defect_types, 1):
        if dataset_type == "mvtec":
            defect_dir = os.path.join(test_dir, defect)
        else:  # VisA
            if '/' in defect:
                parts = defect.split('/')
                defect_dir = os.path.join(test_dir, *parts)
            else:
                defect_dir = os.path.join(test_dir, defect)
        
        defect_images = [f for f in os.listdir(defect_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        if defect_images:
            defect_img_path = os.path.join(defect_dir, random.choice(defect_images))
            defect_img = load_image(defect_img_path)
            
            if defect_img is not None:
                axes[i].imshow(defect_img, cmap='gray')
                axes[i].set_title(defect.split('/')[-1] if '/' in defect else defect)
                axes[i].axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_defect_types_comparison.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Kusur tipleri karşılaştırması kaydedildi: {output_path}")

In [37]:
def visualize_normal_vs_anomaly_samples(data_dir, category, output_dir=RESULTS_DIR, dataset_type="mvtec"):
    """
    Normal ve anomalili görüntüleri yan yana gösterir
    """
    test_dir = os.path.join(data_dir, category, "test")
    
    # Normal örnekler dizini (dataset_type'a göre ayarlama)
    if dataset_type == "mvtec":
        normal_dir = os.path.join(test_dir, "good")
    else:  # VisA
        normal_dir = os.path.join(test_dir, "normal")
        if not os.path.exists(normal_dir):
            normal_dir = os.path.join(test_dir, "good")
    
    # Klasörlerin varlığını kontrol et
    if not os.path.exists(test_dir):
        print(f"UYARI: '{test_dir}' dizini bulunamadı.")
        return
    
    # Normal dizin kontrolü
    if not os.path.exists(normal_dir):
        print(f"UYARI: '{normal_dir}' dizini bulunamadı. Normal dosyaları için test dizini taranacak.")
        
        # Test dizinindeki normal örnekleri bulmak için dosya isimlerine bak
        normal_files = []
        for f in os.listdir(test_dir):
            if f.lower().endswith(('.png', '.jpg', '.jpeg')) and ("normal" in f.lower() or "good" in f.lower()):
                normal_files.append(os.path.join(test_dir, f))
        
        if not normal_files:
            print(f"UYARI: '{category}' için normal örnek bulunamadı.")
            return
        
        # Normal dizini olmadığı için alternatif yaklaşım kullan
        normal_images = []
        normal_files_short = []
        
        for img_path in normal_files:
            img = load_image(img_path)
            if img is not None:
                normal_images.append(img)
                normal_files_short.append(os.path.basename(img_path))
    else:
        # Normal örnekleri yükle
        normal_images = []
        normal_files_short = []
        for img_name in os.listdir(normal_dir):
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            img_path = os.path.join(normal_dir, img_name)
            img = load_image(img_path)
            if img is not None:
                normal_images.append(img)
                normal_files_short.append(img_name)
    
    # Kusur tipleri ve anomali örnekleri
    if dataset_type == "mvtec":
        defect_types = [d for d in os.listdir(test_dir) 
                       if os.path.isdir(os.path.join(test_dir, d)) and d != "good"]
    else:  # VisA
        # VisA'da anomali dizinleri "anomaly", "defect" vb. olabilir
        anomaly_dirs = ["anomaly", "defect", "defective", "abnormal"]
        defect_types = []
        
        for anomaly_dir_name in anomaly_dirs:
            anomaly_dir = os.path.join(test_dir, anomaly_dir_name)
            if os.path.exists(anomaly_dir) and os.path.isdir(anomaly_dir):
                # Alt dizinleri kontrol et (kusur tipleri olabilir)
                sub_dirs = [d for d in os.listdir(anomaly_dir) 
                           if os.path.isdir(os.path.join(anomaly_dir, d))]
                if sub_dirs:
                    # Alt dizinler var, bunlar kusur tipleri
                    for sub_dir in sub_dirs:
                        defect_types.append(os.path.join(anomaly_dir_name, sub_dir))
                else:
                    # Alt dizin yok, doğrudan anomaly dizini
                    defect_types.append(anomaly_dir_name)
    
    if not defect_types:
        print(f"UYARI: '{category}' için kusur tipi bulunamadı.")
        
        # Eğer kusur tipi yoksa, dosya isimlerine göre arama yap
        anomaly_files = []
        for f in os.listdir(test_dir):
            if f.lower().endswith(('.png', '.jpg', '.jpeg')) and any(key in f.lower() for key in 
                                  ["anomaly", "defect", "defective", "abnormal"]):
                anomaly_files.append(os.path.join(test_dir, f))
        
        if not anomaly_files:
            print(f"UYARI: '{category}' için anomali örneği bulunamadı.")
            return
        
        # Anomali örneklerini yükle
        anomaly_images = []
        anomaly_files_short = []
        anomaly_types = []
        
        for img_path in anomaly_files:
            img = load_image(img_path)
            if img is not None:
                anomaly_images.append(img)
                anomaly_files_short.append(os.path.basename(img_path))
                anomaly_types.append("anomaly")
    else:
        # Anomali örneklerini yükle
        anomaly_images = []
        anomaly_files_short = []
        anomaly_types = []
        
        for defect_type in defect_types:
            if dataset_type == "mvtec":
                defect_dir = os.path.join(test_dir, defect_type)
            else:  # VisA
                if '/' in defect_type:
                    parts = defect_type.split('/')
                    defect_dir = os.path.join(test_dir, *parts)
                else:
                    defect_dir = os.path.join(test_dir, defect_type)
            
            if not os.path.isdir(defect_dir):
                continue
                
            for img_name in os.listdir(defect_dir):
                if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue
                img_path = os.path.join(defect_dir, img_name)
                img = load_image(img_path)
                if img is not None:
                    anomaly_images.append(img)
                    anomaly_files_short.append(img_name)
                    anomaly_types.append(defect_type.split('/')[-1] if '/' in defect_type else defect_type)
    
    # Normal ve anomali örnek sayıları
    num_normal = len(normal_images)
    num_anomaly = len(anomaly_images)
    
    if num_normal == 0 or num_anomaly == 0:
        print(f"UYARI: {category} için normal veya anomali görüntüleri bulunamadı.")
        return
    
    # Görselleştirme - Normal ve Anomali örneklerini yan yana göster
    # Her satırda bir normal ve bir anomali örneği
    num_rows = min(5, min(num_normal, num_anomaly))  # En fazla 5 örnek göster
    
    fig, axes = plt.subplots(num_rows, 2, figsize=(10, num_rows * 3))
    fig.suptitle(f"{category} ({dataset_type}) - Normal ve Anomali Örnekleri Karşılaştırması", fontsize=16)
    
    # Tek örnek durumu için
    if num_rows == 1:
        axes = np.array([axes])
    
    for i in range(num_rows):
        # Normal örnek
        normal_idx = i % num_normal
        axes[i, 0].imshow(normal_images[normal_idx], cmap='gray')
        axes[i, 0].set_title(f"Normal: {normal_files_short[normal_idx]}")
        axes[i, 0].axis('off')
        
        # Anomali örneği
        anomaly_idx = i % num_anomaly
        axes[i, 1].imshow(anomaly_images[anomaly_idx], cmap='gray')
        axes[i, 1].set_title(f"Anomali ({anomaly_types[anomaly_idx]}): {anomaly_files_short[anomaly_idx]}")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    output_path = os.path.join(output_dir, f"{category}_{dataset_type}_normal_vs_anomaly_samples.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Normal vs Anomali karşılaştırması kaydedildi: {output_path}")

In [39]:
def process_category(category, data_dir=None, output_dir=RESULTS_DIR, threshold_percentile=95, dataset_type="mvtec"):
    """
    Belirli bir kategori için tüm işlemleri gerçekleştirir
    dataset_type: "mvtec" veya "visa" olabilir, veri seti tipini belirtir
    """
    print(f"\n{'='*50}")
    print(f"İşleniyor: {category} ({dataset_type.upper()} veri seti)")
    print(f"{'='*50}")
    
    # Veri dizini belirtilmemişse, veri setine göre varsayılan değeri kullan
    if data_dir is None:
        if dataset_type == "mvtec":
            data_dir = MVTEC_DIR
        elif dataset_type == "visa":
            data_dir = VISA_DIR
        else:
            raise ValueError(f"Geçersiz veri seti tipi: {dataset_type}")
    
    category_dir = os.path.join(data_dir, category)
    category_output_dir = os.path.join(output_dir, dataset_type, category)
    os.makedirs(category_output_dir, exist_ok=True)
    
    # Veri yolu kontrolü
    if not os.path.exists(category_dir):
        print(f"HATA: {category} kategorisi için veri dizini bulunamadı: {category_dir}")
        return None
    
    # Eğitim ve test verilerini yükle
    try:
        if dataset_type == "mvtec":
            train_images, test_images, test_labels, test_paths, test_types = load_mvtec_data(data_dir, category)
        elif dataset_type == "visa":
            train_images, test_images, test_labels, test_paths, test_types = load_visa_data(data_dir, category)
        else:
            raise ValueError(f"Geçersiz veri seti tipi: {dataset_type}")
    except Exception as e:
        print(f"HATA: {category} kategorisi için veri yüklenirken hata oluştu: {e}")
        return None
    
    # Veri yüklenmediyse uyarı ver ve çık
    if len(train_images) == 0:
        print(f"HATA: {category} kategorisi için eğitim verisi yüklenemedi.")
        return None
    
    if len(test_images) == 0:
        print(f"HATA: {category} kategorisi için test verisi yüklenemedi.")
        return None
    
    # Eğitim ve test verilerini ön işle
    train_processed, test_processed = preprocess_data(train_images, test_images)
    
    # Veri görselleştirme
    try:
        print(f"\n{category} ({dataset_type}) için görselleştirmeler oluşturuluyor...")
        
        # Eğitim örneklerinden bazılarını görselleştir
        if dataset_type == "mvtec":
            train_normal_dir = os.path.join(data_dir, category, "train", "good")
        else:  # VisA
            train_normal_dir = os.path.join(data_dir, category, "train", "normal")
            if not os.path.exists(train_normal_dir):
                train_normal_dir = os.path.join(data_dir, category, "train", "good")
                if not os.path.exists(train_normal_dir):
                    train_normal_dir = os.path.join(data_dir, category, "train")
        
        if os.path.exists(train_normal_dir):
            visualize_samples(train_normal_dir, title=f"{category} ({dataset_type}) - Eğitim Örnekleri", 
                              output_dir=category_output_dir)
        
        # Kusur tiplerini karşılaştır
        visualize_defect_types_comparison(data_dir, category, output_dir=category_output_dir, dataset_type=dataset_type)
        
        # Normal ve anomali örneklerini yan yana karşılaştır
        visualize_normal_vs_anomaly_samples(data_dir, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: Veri görselleştirme sırasında hata oluştu: {e}")
    
    # Modeli oluştur
    input_shape = (IMG_SIZE, IMG_SIZE, 1)  # Gri tonlamalı görüntüler
    model = create_autoencoder(input_shape)
    
    # Model yapısını yazdır
    model.summary()
    
    # Eğitim
    # Dosya uzantısını .keras olarak değiştir (TensorFlow 2.6+ ile uyumlu)
    checkpoint_path = os.path.join(category_output_dir, f"{category}_{dataset_type}_best_model.keras")
    
    start_time = time.time()
    
    checkpointer = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)
    
    # Validasyon için veri setini ayırma
    val_size = int(len(train_processed) * 0.2)
    train_val = train_processed[:-val_size]
    val = train_processed[-val_size:]
    
    history = model.fit(
        train_val, train_val,
        validation_data=(val, val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        callbacks=[checkpointer, early_stopping],
        verbose=1
    )
    
    training_time = time.time() - start_time
    print(f"Eğitim tamamlandı. Süre: {training_time:.2f} saniye")
    
    # En iyi modeli yükle
    try:
        best_model = load_model(checkpoint_path)
        print(f"En iyi model yüklendi: {checkpoint_path}")
    except Exception as e:
        print(f"En iyi model yüklenemedi, hata: {e}")
        print("Mevcut modeli kullanıyoruz.")
        best_model = model
    
    # Eğitim geçmişini görselleştir
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Validasyon Kaybı')
    plt.title(f'{category} ({dataset_type}) - Eğitim/Validasyon Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    output_path = os.path.join(category_output_dir, f"{category}_{dataset_type}_training_history.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Eğitim geçmişi grafiği kaydedildi: {output_path}")
    
    # Test ve anomali tespiti
    reconstructed, mse, predictions, threshold, accuracy, precision, recall, f1, roc_auc = detect_anomalies(
        test_processed, test_labels, best_model, threshold_percentile)
    
    # Sonuçları görselleştir
    print(f"\n{category} ({dataset_type}) için analiz sonuçları görselleştiriliyor...")
    
    try:
        # Rekonstrüksiyon sonuçları
        visualize_reconstruction_results(test_processed, test_labels, reconstructed, mse, predictions, threshold, 
                                       test_types, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: Rekonstrüksiyon görselleştirme hatası: {e}")
    
    try:
        # Normal vs Anomali MSE karşılaştırması
        compare_normal_vs_anomaly(test_processed, test_labels, reconstructed, mse, threshold, category, 
                                output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: MSE karşılaştırma görselleştirme hatası: {e}")
    
    try:
        # ROC eğrisi
        visualize_roc_curve(test_labels, mse, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: ROC eğrisi görselleştirme hatası: {e}")
    
    try:
        # Karışıklık matrisi
        visualize_confusion_matrix(test_labels, predictions, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: Karışıklık matrisi görselleştirme hatası: {e}")
    
    try:
        # Detaylı kusur analizi
        visualize_detailed_comparison(test_processed, test_labels, reconstructed, mse, predictions, threshold, 
                                     test_types, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: Detaylı kusur analizi görselleştirme hatası: {e}")
    
    try:
        # Kusur tiplerine göre istatistikler
        visualize_all_defect_stats(test_processed, test_labels, mse, predictions, threshold, 
                                  test_types, category, output_dir=category_output_dir, dataset_type=dataset_type)
    except Exception as e:
        print(f"UYARI: Kusur tipi istatistikleri görselleştirme hatası: {e}")
    
    # Sonuçları döndür
    results = {
        'category': category,
        'dataset_type': dataset_type,  # Veri seti tipini ekle
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'threshold': threshold,
        'normal_count': np.sum(test_labels == 0),
        'anomaly_count': np.sum(test_labels == 1),
        'training_time': training_time
    }
    
    # Sonuçları metin dosyasına kaydet
    with open(os.path.join(category_output_dir, f"{category}_{dataset_type}_results.txt"), "w") as f:
        f.write(f"Kategori: {category} ({dataset_type})\n")
        f.write(f"Doğruluk (Accuracy): {accuracy:.4f}\n")
        f.write(f"Hassasiyet (Precision): {precision:.4f}\n")
        f.write(f"Duyarlılık (Recall): {recall:.4f}\n")
        f.write(f"F1 Skoru: {f1:.4f}\n")
        f.write(f"ROC AUC: {roc_auc:.4f}\n")
        f.write(f"Eşik Değeri: {threshold:.6f}\n")
        f.write(f"Normal Örnek Sayısı: {np.sum(test_labels == 0)}\n")
        f.write(f"Anomali Örnek Sayısı: {np.sum(test_labels == 1)}\n")
        f.write(f"Eğitim Süresi: {training_time:.2f} saniye\n")
    
    print(f"Sonuçlar başarıyla kaydedildi: {category_output_dir}")
    return results

In [41]:
def main():
    """Ana program fonksiyonu"""
    # Sonuçlar dizini
    os.makedirs(RESULTS_DIR, exist_ok=True)
    print(f"Sonuçlar '{RESULTS_DIR}' dizinine kaydedilecek.")
    
    # İşlenecek kategorileri belirle
    all_results = []
    
    # MVTec veri seti kategorileri
    if USE_MVTEC:
        print("\n=== MVTec Veri Seti İşleniyor ===")
        mvtec_out_dir = os.path.join(RESULTS_DIR, "mvtec")
        os.makedirs(mvtec_out_dir, exist_ok=True)
        
        for category in MVTEC_CATEGORIES:
            try:
                results = process_category(category, data_dir=MVTEC_DIR, output_dir=RESULTS_DIR, dataset_type="mvtec")
                if results:
                    all_results.append(results)
                    print(f"MVTec: {category} kategorisi başarıyla işlendi.")
                else:
                    print(f"MVTec: {category} kategorisi işleme başarısız.")
            except Exception as e:
                print(f"HATA: MVTec {category} kategorisi işlenirken bir hata oluştu: {e}")
    
    # VisA veri seti kategorileri
    if USE_VISA:
        print("\n=== VisA Veri Seti İşleniyor ===")
        visa_out_dir = os.path.join(RESULTS_DIR, "visa")
        os.makedirs(visa_out_dir, exist_ok=True)
        
        for category in VISA_CATEGORIES:
            try:
                results = process_category(category, data_dir=VISA_DIR, output_dir=RESULTS_DIR, dataset_type="visa")
                if results:
                    all_results.append(results)
                    print(f"VisA: {category} kategorisi başarıyla işlendi.")
                else:
                    print(f"VisA: {category} kategorisi işleme başarısız.")
            except Exception as e:
                print(f"HATA: VisA {category} kategorisi işlenirken bir hata oluştu: {e}")
    
    # Tüm kategorilerin performansını karşılaştır
    if all_results:
        print("\nTüm kategorilerin karşılaştırmalı analizi yapılıyor...")
        try:
            # Tüm sonuçları birlikte karşılaştır
            visualize_all_categories_comparison(all_results)
            
            # MVTec ve VisA sonuçlarını ayrı ayrı karşılaştır
            mvtec_results = [r for r in all_results if r.get('dataset_type') == 'mvtec']
            visa_results = [r for r in all_results if r.get('dataset_type') == 'visa']
            
            if mvtec_results and len(mvtec_results) > 1:
                visualize_all_categories_comparison(
                    mvtec_results, 
                    output_path=os.path.join(RESULTS_DIR, 'mvtec_performance_comparison.png'),
                    title='MVTec Kategorileri Performans Karşılaştırması'
                )
            
            if visa_results and len(visa_results) > 1:
                visualize_all_categories_comparison(
                    visa_results, 
                    output_path=os.path.join(RESULTS_DIR, 'visa_performance_comparison.png'),
                    title='VisA Kategorileri Performans Karşılaştırması'
                )
            
            # Sonuçları bir CSV dosyasına kaydet
            import csv
            csv_path = os.path.join(RESULTS_DIR, "all_results.csv")
            with open(csv_path, "w", newline='') as csvfile:
                fieldnames = ['category', 'dataset_type', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 
                              'normal_count', 'anomaly_count', 'training_time']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                
                writer.writeheader()
                for result in all_results:
                    writer.writerow({k: result[k] for k in fieldnames if k in result})
            
            print(f"Tüm sonuçlar CSV dosyasına kaydedildi: {csv_path}")
        except Exception as e:
            print(f"UYARI: Karşılaştırmalı analiz sırasında hata oluştu: {e}")
    else:
        print("UYARI: Hiç sonuç üretilemedi, karşılaştırmalı analiz yapılamadı.")
    
    print("\nTüm işlemler tamamlandı. Sonuçları kontrol ediniz.")


if __name__ == "__main__":
    main()

Sonuçlar 'anomaly_detection_results' dizinine kaydedilecek.

=== MVTec Veri Seti İşleniyor ===

İşleniyor: hazelnut (MVTEC veri seti)
Eğitim dizini: C:\Users\Tansu\OneDrive\Masaüstü\mvtec\hazelnut\train
Test dizini: C:\Users\Tansu\OneDrive\Masaüstü\mvtec\hazelnut\test
Eğitim 'good' dizini: C:\Users\Tansu\OneDrive\Masaüstü\mvtec\hazelnut\train\good
Eğitim 'good' dizinindeki dosya sayısı: 391


hazelnut: Eğitim verilerini yükleme: 100%|██████████| 391/391 [00:12<00:00, 30.60it/s]


Test 'good' dizini: C:\Users\Tansu\OneDrive\Masaüstü\mvtec\hazelnut\test\good
Test 'good' dizinindeki dosya sayısı: 40


hazelnut: Normal test verilerini yükleme: 100%|██████████| 40/40 [00:01<00:00, 30.46it/s]


Tespit edilen kusur tipleri: ['crack', 'cut', 'hole', 'print']
Kusur dizini: C:\Users\Tansu\OneDrive\Masaüstü\mvtec\hazelnut\test\crack
Kusur dizinindeki dosya sayısı: 18


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_105 (Conv2D)             │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_106 (Conv2D)             │ (None, 64, 64, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_107 (Conv2D)             │ (None, 32, 32, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_42 (UpSampling2D) │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_108 (Conv2D)             │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_43 (UpSampling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_109 (Conv2D)             │ (None, 128, 128, 1)    │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.1652
Epoch 1: val_loss improved from inf to 0.12737, saving model to anomaly_detection_results\visa\pipe_fryum\pipe_fryum_visa_best_model.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - loss: 0.1644 - val_loss: 0.1274
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.1043
Epoch 2: val_loss improved from 0.12737 to 0.03796, saving model to anomaly_detection_results\visa\pipe_fryum\pipe_fryum_visa_best_model.keras
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - loss: 0.1021 - val_loss: 0.0380
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.0383
Epoch 3: val_loss did not improve from 0.03796
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - loss: 0.0383 - val_loss: 0.0389
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.0390
Epoch 4: val_loss did not improve from 0.03796
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - loss: 0.0390 - val_loss: 0.0390
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.0393
Epoch 5